In [134]:
import time, subprocess, os
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from utils import clear_sessions, check_and_run_docker, get_proxies, check_container

In [193]:
country = 'uk'
company_block_css = "#service-providers > div > div.directory-list > ul > li.firm-wrapper"
company_name_css = "div.firm-header > div > h3 > a::text"
website_link_css = "div.firm-header > a::attr(href)"
next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
listing_website_start_url = "https://www.goodfirms.co/directory/cms/top-website-development-companies?locations%5B166%5D=gb&page=26"
page_num = 26
chromedriver_command_executor = "http://localhost:4444/wd/hub"
folder_name = "goodfirms_webdev_usuk"
is_next_page_url_in_format = False

In [194]:
# country = 'uk'
# company_block_css = "#js-listing-app > ul > li.agency-list-item.agency-item"
# company_name_css = "div > div.agency-item--inner > div.agency-item--main > div > div.info-head > h3 > a::text"
# website_link_css = "div > div.agency-item--actions > a::attr(href)"
# next_page_css = "#paginator > ul > li.page-item.active+li>a"
# is_next_page_url_in_format = True
# next_page_url = "https://www.designrush.com/agency/web-development-companies/uk"
# listing_website_start_url = "https://www.designrush.com/agency/web-development-companies/uk"
# page_num = 1
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "designrush_webdev_usuk"

In [195]:
# country = 'uk'
# company_block_css = ".directory-providers__list >li.provider-card"
# company_name_css = "div.provider-header > h3 > a::attr(title)"
# website_link_css = "div.provider-card__body.provider-body > div.provider-card__cta > a.provider-card__visit-btn.provider-visit.track-website-visit::attr(href)"
# next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
# listing_website_start_url = "https://themanifest.com/uk/web-development/companies"
# page_num = 0
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "temp"
# is_next_page_url_in_format = False

In [196]:
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists(f"data/{folder_name}"):
    os.makedirs(f"data/{folder_name}")
    os.makedirs(f"data/{folder_name}/emails")
    os.makedirs(f"data/{folder_name}/logs")
    os.makedirs(f"data/{folder_name}/out")

In [197]:
def run_crawl(full_path, page_num):
    subprocess.Popen(f'''cd {os.getcwd()} && \
                    scrapy crawl fetch_emails \
                    -a start_url=file://{full_path} \
                    -a country={country} \
                    -a company_block_css="{company_block_css}" \
                    -a company_name_css="{company_name_css}" \
                    -a website_link_css="{website_link_css}" \
                    -o data/{folder_name}/emails/{folder_name}{country}{page_num}.csv \
                    --logfile data/{folder_name}/logs/{folder_name}{country}{page_num}.log''',
                    shell=True)

In [198]:
def get_companies(url_to_scrape=listing_website_start_url, page_num=page_num):
    # Set the options for the webdriver
    # PROXIES = get_proxies()
    options = Options()
    next_page = url_to_scrape
    driver = None
    try:
        signal = True
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # options.add_argument("--headless")

        # Set the proxy
        # random_proxy = random.choice(PROXIES)
        # print("Proxy in use: %s" % random_proxy)
        # options.add_argument('--proxy-server=%s' % random_proxy)
        # options.add_argument('--proxy-server=%s' % '104.234.157.100:21278')

        driver = webdriver.Remote(
            command_executor=chromedriver_command_executor,
            options=options,
        )
        with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
            while next_page:
                print(next_page)
                driver.get(next_page)
                print(driver.title)
                file_path = f'data/{folder_name}/out/{folder_name}{country}{page_num}.html'
                full_path = os.path.abspath(file_path)
                with open(file_path, 'w') as f:
                    f.write(driver.page_source)
                executor.submit(run_crawl, full_path, page_num)
                if is_next_page_url_in_format:
                    next_page = next_page_url + f"?page={page_num+1}"
                else:
                    next_page_element = driver.find_element(By.CSS_SELECTOR, next_page_css)
                    if next_page_element:
                        next_page = next_page_element.get_attribute('href')
                    else:
                        next_page = None
                    print(next_page)
                page_num += 1

    
    except NoSuchElementException as e:
        print("No Such Element Exception")
        print(e)
        signal = False
    except WebDriverException as e:
        signal = (page_num, next_page)
        print(e)
    except Exception as e:
        print(e)
        print(e.__class__.__name__)
        signal = False
    finally:
        if driver:
            driver.close()
        clear_sessions()
        return signal


In [199]:
check_and_run_docker()
time.sleep(2)
clear_sessions()

Docker container already running


In [200]:
clear_sessions()
signal = get_companies(listing_website_start_url, page_num)
print(signal)
while isinstance(signal, tuple):
    signal = get_companies(signal[1],signal[0])
    print(signal)
    time.sleep(10)

https://www.goodfirms.co/directory/cms/top-website-development-companies?locations%5B166%5D=gb&page=26
Top 10+ Web Development Companies - Jan 2024 Reviews | GoodFirms
No Such Element Exception
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"}
  (Session info: chrome=120.0.6099.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55bca8a74f83 <unknown>
#1 0x55bca872dcf7 <unknown>
#2 0x55bca877d99e <unknown>
#3 0x55bca877daa1 <unknown>
#4 0x55bca87c8d64 <unknown>
#5 0x55bca87a70dd <unknown>
#6 0x55bca87c6006 <unknown>
#7 0x55bca87a6e53 <unknown>
#8 0x55bca876edd4 <unknown>
#9 0x55bca87701de <unknown>
#10 0x55bca8a39531 <unknown>
#11 0x55bca8a3d455 <unknown>
#12 0x55bca8a25f55 <unknown>
#13 0x55bca8a3e0ef <unknown>
#14 0x55bca8a0999f <unknow